In [ ]:
!pip install transformers datasets sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer, RobertaTokenizer

model_name = "Salesforce/codet5-small"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
train_data = [
    {"input": "Get all employees in the IT department", "output": "SELECT * FROM employees WHERE department = 'IT';"},
    {"input": "Get all employees in the CSE department", "output": "SELECT * FROM employees WHERE department = 'CSE';"},
    {"input": "Find all employees who joined after 2020", "output": "SELECT * FROM employees WHERE joining_date > '2020-01-01';"},
    {"input": "Find products that cost more than 50000", "output": "SELECT * FROM products WHERE price > 50000;"},
    {"input": "Retrieve all orders placed in 2023", "output": "SELECT * FROM orders WHERE order_date >= '2023-01-01';"},
    {"input": "Get employees earning more than 100000", "output": "SELECT * FROM employees WHERE salary > 100000;"},
]


In [ ]:
def format_data(data):
    inputs = []
    outputs = []

    for pair in data:
        inputs.append("translate English to SQL: " + pair["input"])
        outputs.append(pair["output"])

    return inputs, outputs

train_inputs, train_outputs = format_data(train_data)


In [ ]:
input_encodings = tokenizer(train_inputs, padding=True, truncation=True, return_tensors="pt", max_length=512)
output_encodings = tokenizer(train_outputs, padding=True, truncation=True, return_tensors="pt", max_length=512)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class SQLDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels["input_ids"][idx],
        }

train_dataset = SQLDataset(input_encodings, output_encodings)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [ ]:
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}: Loss {loss.item()}")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1: Loss 3.3197460174560547


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Epoch 2: Loss 2.375563859939575
Epoch 3: Loss 2.689115047454834
Epoch 4: Loss 1.353337049484253
Epoch 5: Loss 1.0560028553009033
Epoch 6: Loss 1.080121397972107
Epoch 7: Loss 1.148177146911621
Epoch 8: Loss 0.8597342371940613
Epoch 9: Loss 0.582282543182373
Epoch 10: Loss 0.5349816083908081
Epoch 11: Loss 0.2187470644712448
Epoch 12: Loss 0.03851495310664177
Epoch 13: Loss 0.09599349647760391
Epoch 14: Loss 0.09161500632762909
Epoch 15: Loss 0.05175216123461723
Epoch 16: Loss 0.04830288141965866
Epoch 17: Loss 0.03914123773574829
Epoch 18: Loss 0.09110500663518906
Epoch 19: Loss 0.037260107696056366
Epoch 20: Loss 0.03265184909105301
Epoch 21: Loss 0.012774670496582985
Epoch 22: Loss 0.010270101949572563
Epoch 23: Loss 0.019425619393587112
Epoch 24: Loss 0.02915034070611
Epoch 25: Loss 0.01696822978556156
Epoch 26: Loss 0.01616232469677925
Epoch 27: Loss 0.01927558146417141
Epoch 28: Loss 0.0331563875079155
Epoch 29: Loss 0.014665360562503338
Epoch 30: Loss 0.019795730710029602
Epoch 3

In [ ]:
def generate_sql(query):
    input_text = "Convert the following natural language query to SQL: " + query

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)

    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query


In [ ]:
test_query = "Give me all products that cost more than 60000"    #Find products that cost more than 50000
print("Generated SQL:", generate_sql(test_query))

Generated SQL: SELECT * FROM products WHERE price > 60000;


In [ ]:
test_query = "Give all orders placed in 2024"         #Retrieve all orders placed in 2023
print("Generated SQL:", generate_sql(test_query))

Generated SQL: SELECT * FROM orders WHERE order_date >= '2024-01-01';


In [ ]:
test_query = "Get the list of employees getting more than 250000 salary"    #Get employees earning more than 100000
print("Generated SQL:", generate_sql(test_query))

Generated SQL: SELECT * FROM employees WHERE salary > 250000;


In [ ]:
test_query= "Give me all the employees who are in the CSE-AIML department"      #Get all employees in the IT department
print("Generated SQL:", generate_sql(test_query))

Generated SQL: SELECT * FROM employees WHERE department = 'CSE-AIML';
